In [1]:
## Import libraries
import requests
import pandas as pd
import pymysql
from sqlalchemy import create_engine

In [2]:
## Connect to MySQL DB - CLOUD   
driver   = 'mysql+pymysql:'
user     = #####
password = #####
ip       = '35.195.172.42' 
database = 'country_battle' 

connection_string = f'{driver}//{user}:{password}@{ip}/{database}'
engine = create_engine(connection_string)

# Countries CSV

Create a table with a list of the world countries

In [3]:
### https://www.kaggle.com/juanumusic/countries-iso-codes
countries = pd.read_csv('../datasets/countries-iso-codes/wikipedia-iso-country-codes.csv')
original_countries = countries.copy()
countries

,English short name lower case,Alpha-2 code,Alpha-3 code,Numeric code,ISO 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL
3,Algeria,DZ,DZA,12,ISO 3166-2:DZ
4,American Samoa,AS,ASM,16,ISO 3166-2:AS
...,...,...,...,...,...
241,Wallis and Futuna,WF,WLF,876,ISO 3166-2:WF
242,Western Sahara,EH,ESH,732,ISO 3166-2:EH
243,Yemen,YE,YEM,887,ISO 3166-2:YE
244,Zambia,ZM,ZMB,894,ISO 3166-2:ZM


In [4]:
## Clean table
countries.drop(['Alpha-3 code', 'Numeric code', 'ISO 3166-2'], axis = 1, inplace = True)
countries.columns = ['country_name', 'alpha_2_code']

In [5]:
## Export it to MySQL
countries.to_sql('country', con=engine, if_exists = 'append', index = False) 

C:\Users\miner\Anaconda3\lib\site-packages\pymysql\cursors.py:170: Warning: (1366, "Incorrect string value: '\\xE1ndar ...' for column 'VARIABLE_VALUE' at row 1")
  result = self._query(query)


In [6]:
##Import it back with the table PK
countries_with_id = pd.read_sql('country', engine)
print('Len countries after DB: ', len(countries_with_id))

Len countries after DB:  246


# Spotify CSV

Create tables: track, artist and genre. Create track-genre relationship table.

### Import raw CSV

In [7]:
### https://www.kaggle.com/zaheenhamidani/ultimate-spotify-tracks-db
raw_data = pd.read_csv('../datasets/ultimate_spotify_tracks_db/SpotifyFeatures.csv')
original_raw_data = pd.read_csv('../datasets/ultimate_spotify_tracks_db/SpotifyFeatures.csv')

In [8]:
raw_data.head(3)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.0,C#,0.346,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.0,F#,0.151,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.0,C,0.103,-13.879,Minor,0.0362,99.488,5/4,0.368


In [9]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232725 entries, 0 to 232724
Data columns (total 18 columns):
genre               232725 non-null object
artist_name         232725 non-null object
track_name          232725 non-null object
track_id            232725 non-null object
popularity          232725 non-null int64
acousticness        232725 non-null float64
danceability        232725 non-null float64
duration_ms         232725 non-null int64
energy              232725 non-null float64
instrumentalness    232725 non-null float64
key                 232725 non-null object
liveness            232725 non-null float64
loudness            232725 non-null float64
mode                232725 non-null object
speechiness         232725 non-null float64
tempo               232725 non-null float64
time_signature      232725 non-null object
valence             232725 non-null float64
dtypes: float64(9), int64(2), object(7)
memory usage: 32.0+ MB


In [10]:
raw_data.columns

Index(['genre', 'artist_name', 'track_name', 'track_id', 'popularity',
       'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'key', 'liveness', 'loudness', 'mode',
       'speechiness', 'tempo', 'time_signature', 'valence'],
      dtype='object')

### Explore CSV normalization needs

In [11]:
## Check if each genre is unique
unique_genres = raw_data.genre.unique()
print(unique_genres)

['Movie' 'R&B' 'A Capella' 'Alternative' 'Country' 'Dance' 'Electronic'
 'Anime' 'Folk' 'Blues' 'Opera' 'Hip-Hop' "Children's Music"
 'Children’s Music' 'Rap' 'Indie' 'Classical' 'Pop' 'Reggae' 'Reggaeton'
 'Jazz' 'Rock' 'Ska' 'Comedy' 'Soul' 'Soundtrack' 'World']


In [12]:
## Unify values 'Children's Music' and 'Children’s Music' in genre column
## Also in the original data as this is not a manipulation of data but a correction
raw_data['genre'].replace('Children’s Music', 'Children\'s Music', inplace = True)
original_raw_data['genre'].replace('Children’s Music', 'Children\'s Music', inplace = True)
print('Raw data DF length after genre replace: ', len(raw_data))

Raw data DF length after genre replace:  232725


In [13]:
## Normalize all artist names -- we will titlecase them as a first mesasure to ensure that different spellings don't 
## create duplicates
raw_data['artist_name'] = raw_data['artist_name'].apply(lambda x : x.title())
original_raw_data['artist_name'] = original_raw_data['artist_name'].apply(lambda x : x.title())
print('Raw data DF length after genre replace: ', len(raw_data))

Raw data DF length after genre replace:  232725


In [14]:
## Normalize all tracks names -- we will titlecase them as a first mesasure to ensure that different spellings don't 
## create duplicates
raw_data['track_name'] = raw_data['track_name'].apply(lambda x : x.title())
original_raw_data['track_name'] = original_raw_data['track_name'].apply(lambda x : x.title())
print('Raw data DF length after genre replace: ', len(raw_data))

Raw data DF length after genre replace:  232725


### Create artists table

In [15]:
## Extract series 'artist_name' from the raw data
artists = raw_data['artist_name'].copy()
print('Len artist name: ', len(artists))

Len artist name:  232725


In [16]:
## Delete duplicated values
artists.drop_duplicates(keep='first', inplace=True)
print('Len artist name unique: ', len(artists))

Len artist name unique:  14561


In [17]:
## Export it to MySQL
artists.to_sql('artist', con=engine, if_exists = 'append', index = False) 

In [18]:
##Import it back with the table PK
artists_with_id = pd.read_sql('artist', engine)
print('Len artists after DB: ', len(artists_with_id))

Len artists after DB:  14561


In [19]:
##Add the artist_id to the raw table
raw_with_artist_id = pd.merge(raw_data, artists_with_id, on='artist_name')

In [20]:
##Change name of artist_id column
raw_with_artist_id = raw_with_artist_id.rename(columns={'ID':'artist_id'})

In [21]:
##Check changes on the raw_table
raw_with_artist_id.head(3)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_id
0,Movie,Henri Salvador,C'Est Beau De Faire Un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.9100,0.0,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,1
1,Movie,Henri Salvador,Dis-Moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.3260,0.0,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,1
2,Movie,Henri Salvador,Le Petit Souper Aux Chandelles,0Mf1jKa8eNAf1a4PwTbizj,0,0.749,0.578,160627,0.0948,0.0,C#,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358,1


In [22]:
print('Raw data DF length after adding artist ID: ', len(raw_with_artist_id))

Raw data DF length after adding artist ID:  232725


### Create genre table

In [23]:
## Extract series 'genre' from the raw data
genres = raw_data['genre'].copy()
print('Len genres: ', len(genres))

Len genres:  232725


In [24]:
## Delete duplicated values
genres.drop_duplicates(keep='first', inplace=True)
print('Len genres unique: ', len(genres))

Len genres unique:  26


In [25]:
## Change name of column 'name' to 'genre_name'
genres.name = 'genre_name' 

In [26]:
## Export it to MySQL
genres.to_sql('genre', con=engine, if_exists = 'append', index = False) 

In [27]:
##Import it back with the table PK
genres_with_id = pd.read_sql('genre', engine)
print('Len genres after DB: ', len(genres_with_id))

Len genres after DB:  26


In [28]:
genres_with_id = genres_with_id.rename(columns={'genre_name':'genre'})
genres_with_id.head(3)

,ID,genre
0,1,Movie
1,2,R&B
2,3,A Capella


In [29]:
##Add the genre_id to the raw table
raw_with_artist_genre_id = pd.merge(raw_with_artist_id, genres_with_id, on='genre')

In [30]:
##Change name of genre_id column
raw_with_artist_genre_id = raw_with_artist_genre_id.rename(columns={'ID':'genre_id'})
raw_with_artist_genre_id.head(3)

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,artist_id,genre_id
0,Movie,Henri Salvador,C'Est Beau De Faire Un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.9100,0.0,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,1,1
1,Movie,Henri Salvador,Dis-Moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.3260,0.0,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,1,1
2,Movie,Henri Salvador,Le Petit Souper Aux Chandelles,0Mf1jKa8eNAf1a4PwTbizj,0,0.749,0.578,160627,0.0948,0.0,C#,0.1070,-14.970,Major,0.1430,87.479,4/4,0.358,1,1


In [31]:
print('Raw data DF length after adding genre ID: ', len(raw_with_artist_genre_id))

Raw data DF length after adding genre ID:  232725



### Create track table

Track table should include all the information on the tracks + the artist ID as a FK

In [32]:
## We will create a copy of the original data to work, so we ensure that the records have not been altered.
raw_data_2 = original_raw_data.copy()

In [33]:
##Check duplicates. Track name is not optimal, as we can assume that some tracks may have the same title, but be different
##songs, by different artists. Luckily, we have the Spotify ID, which will identify each song uniquely. 

##Count unique values on track_id column
len(raw_data_2.track_id.unique())

176774

#### Clean duplicates: drop genre

In [34]:
## First intuition will be to consider that a song can belong to many genres, therefore creating several rows in the CSV.

## Drop genre column and drop duplicates after not considering that column
raw_data_no_genre = raw_data_2[raw_data_2.columns[1:]].copy()
raw_data_no_genre = raw_data_no_genre.drop_duplicates()
len(raw_data_no_genre)

191049

#### Clean duplicates: average popularity

In [35]:
##The rest of attributes should be always the same for the same unique song (i.e.: a song can't have two different 'energy'
## values). We can check which IDs are repeated in the track_id column no perform a manual inspection that can provide more 
##insight:

##Count how many times each ID is repeated in the track_id column and create a list with the ones that are repeated more than 
##once:
repeated_track_ids = pd.DataFrame(raw_data_no_genre.track_id.value_counts()).reset_index().copy()
repeated_track_ids

,index,track_id
0,2QOcDtnLi8V3aBjq4m7dbI,3
1,16DhvbuyvJob4Q9GHNYu2n,3
2,4dVbhS6OiYvFikshyaQaCN,3
3,5HPnumlogzZtpLEaORZjz1,3
4,6kayAD8DHrHe1bn5Russar,3
...,...,...
176769,7rJPkmJs8zkdyGJzkjdCif,1
176770,75yRHbCeR5tY00X9vLNRuj,1
176771,6dcgcciPlpMMsJ1mzxnjHM,1
176772,4DCKfM7M9G7dIeaqrGqpSs,1


In [36]:
repeated_track_ids = repeated_track_ids[repeated_track_ids['track_id'] > 1].copy()
repeated_track_ids

,index,track_id
0,2QOcDtnLi8V3aBjq4m7dbI,3
1,16DhvbuyvJob4Q9GHNYu2n,3
2,4dVbhS6OiYvFikshyaQaCN,3
3,5HPnumlogzZtpLEaORZjz1,3
4,6kayAD8DHrHe1bn5Russar,3
...,...,...
12629,5ytmV00jr7di9K0O10dkBO,2
12630,5ODOI1L5b65QrDNBGVpRvJ,2
12631,3ti4eDm3s0dGWxQKeJ1OeM,2
12632,2JuasWPUodaUxf5nwNpciQ,2


In [37]:
##This represents less than 5,5% of the data. We perform a manual inspection to understand a little more:
repeated_track_ids.head(20)

,index,track_id
0,2QOcDtnLi8V3aBjq4m7dbI,3
1,16DhvbuyvJob4Q9GHNYu2n,3
2,4dVbhS6OiYvFikshyaQaCN,3
3,5HPnumlogzZtpLEaORZjz1,3
4,6kayAD8DHrHe1bn5Russar,3
5,2XEdrmqNM9ZQQ24ViuceB0,3
6,2Y0iGXY6m6immVb2ktbseM,3
7,4Ahphc0UzRSoMpJRHgnGQK,3
8,7Ml1vXT6LaH34jVBGYygWY,3
9,7lLAOmWJXM9csJkg3yKrph,3


In [38]:
raw_data_no_genre[raw_data_no_genre['track_id'] == '7tX9wItFwaTPCuVBneLrHs']

,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
10367,Jhene Aiko,You Are Here,7tX9wItFwaTPCuVBneLrHs,46,0.187,0.376,219520,0.699,0.00244,E,0.33,-7.635,Minor,0.0745,141.085,4/4,0.133
20203,Jhene Aiko,You Are Here,7tX9wItFwaTPCuVBneLrHs,50,0.187,0.376,219520,0.699,0.00244,E,0.33,-7.635,Minor,0.0745,141.085,4/4,0.133
59688,Jhene Aiko,You Are Here,7tX9wItFwaTPCuVBneLrHs,51,0.187,0.376,219520,0.699,0.00244,E,0.33,-7.635,Minor,0.0745,141.085,4/4,0.133


In [39]:
## The attribute that seems to change is popularity. The rest of the parameters seem to be kept the same. After some
## consideration on the way to proceed, we decide to unify these records and calculate the mean popularity among the 
## different records of the same track, because: a) in case we need these tracks for our analysis, all the other parameters 
## can still be considered valid. b) since we will keep a list of the tracks that are being modified, we will be able to 
## control how this biased data may affect our data (if it does.)

In [40]:
## Group by Spotify ID (and other text columns) and mean the rest of columns
raw_data_for_tracks_grouped = raw_data_no_genre.groupby(['track_id','track_name', 'artist_name'], as_index=False).mean()

In [41]:
len(raw_data_for_tracks_grouped)

176893

In [42]:
## This number is still higher than the amount of unique spotify IDs. Let's perform a second iteration searching for
##duplicated values on the table.
repeated_track_ids_2 = pd.DataFrame(raw_data_for_tracks_grouped.track_id.value_counts()).reset_index().copy()
repeated_track_ids_2

,index,track_id
0,3yawCOFKiw8TVVdCubX4hu,2
1,5yQ9iMZXGcr5rlO4hoLsP4,2
2,0ritgEzDOsxbd2IfYX96S9,2
3,4tggZyGJMNJMooWuRbvN5q,2
4,1FTBGFF2CJIEsMdkRMHo6z,2
...,...,...
176769,0MVUm3Nied9ocRAkDjRWvP,1
176770,6EKR5o7FNosJP3UwZgv7bG,1
176771,6USfUUlSFATxDSrSGG2xNS,1
176772,2KnTNMXc5aSpaiijRiNWlh,1


In [43]:
repeated_track_ids_2 = repeated_track_ids_2[repeated_track_ids_2['track_id'] > 1].copy()
repeated_track_ids_2.head(20)

,index,track_id
0,3yawCOFKiw8TVVdCubX4hu,2
1,5yQ9iMZXGcr5rlO4hoLsP4,2
2,0ritgEzDOsxbd2IfYX96S9,2
3,4tggZyGJMNJMooWuRbvN5q,2
4,1FTBGFF2CJIEsMdkRMHo6z,2
5,2g9EvzLR2sYscJQubwKGqu,2
6,6rx4jBTSagd1W4ZkQI63oK,2
7,2g4SumkFZpHOypfF5xng9Z,2
8,4NqxAPv6ZAg9Lccf1QFTVg,2
9,4MYb7NWLwXNDB7bYs3HeX8,2


In [44]:
raw_data_for_tracks_grouped[raw_data_for_tracks_grouped['track_id'] == '4WUduEPZZG6wiE9B5IklnY']

,track_id,track_name,artist_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
102924,4WUduEPZZG6wiE9B5IklnY,Let Me In,Grouplove,58.5,0.00397,0.476,239658.0,0.744,0.000173,0.0982,-6.602,0.0515,127.934,0.159
102925,4WUduEPZZG6wiE9B5IklnY,Let Me In - Soundtrack Version,Grouplove,55.0,0.00397,0.476,239658.0,0.744,0.000173,0.0982,-6.602,0.0515,127.934,0.159


In [45]:
## After all normalization, there are still 119 duplicates, which account for less than 0,07% of the data. The normalization of
## these tracks will require a manual exploration to decide the action to take in this case. For the purpose of our current
## study, this level of detail won't generate significant biases on the results. Therefore we decide to keep these duplicates.
len(raw_data_for_tracks_grouped) - len(raw_data_2.track_id.unique())

119

#### Create final track table: 

In [46]:
##Add the artist_id to the  table
tracks_table = pd.merge(raw_data_for_tracks_grouped, artists_with_id, on='artist_name')

In [47]:
## Create a table only with the columns that we want to keep on the track table
tracks = tracks_table[['track_name', 'track_id', 'popularity','acousticness', 'danceability', 'duration_ms', 
        'energy', 'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'ID']]

In [48]:
##Change the names to meet the DB columns
tracks.columns = ['track_name', 'spotify_id', 'popularity', 'acousticness', 'danceability', 'duration_ms', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence', 'artist_id']

In [49]:
## Export it to MySQL
tracks.to_sql('track', con=engine, if_exists = 'append', index = False)

In [50]:
##Import it back with the table PK
tracks_with_id = pd.read_sql('track', engine)
print('Len tracks after DB: ', len(tracks_with_id))

Len tracks after DB:  176893


### Create track_genre relationship table

In [51]:
##Create a table with Spotify_id + track_id
track_id_by_spotify_id_minimal = tracks_with_id[['ID', 'spotify_id']]
track_id_by_spotify_id_minimal = track_id_by_spotify_id_minimal.rename(columns={'ID':'track_id'})
len(track_id_by_spotify_id_minimal)

176893

In [52]:
##Create a table with Spotify_id + genre_id
genre_id_by_spotify_id_minimal = raw_with_artist_genre_id[['track_id', 'genre_id']]
genre_id_by_spotify_id_minimal = genre_id_by_spotify_id_minimal.rename(columns={'track_id':'spotify_id'})

In [53]:
len(genre_id_by_spotify_id_minimal)

232725

In [54]:
##Create the relation table. The difference in rows is caused by the existing duplicates that we have detected in the tracks
##table already
genre_id_track_id = pd.merge(genre_id_by_spotify_id_minimal, track_id_by_spotify_id_minimal, on='spotify_id')
genre_id_track_id.drop(columns=['spotify_id'], inplace = True)
len(genre_id_track_id)

233097

In [55]:
## Export it to MySQL
genre_id_track_id.to_sql('track_genre', con=engine, if_exists = 'append', index = False)

In [56]:
##Import it back with the table PK
track_genre_with_id = pd.read_sql('track_genre', engine)
print('Len track-genre table after DB: ', len(track_genre_with_id))

Len track-genre table after DB:  233097


# Last FM API dataset integration

### Import and normalize artist table from DB

In [57]:
##Read table
artists_table = pd.read_sql('artist', engine)

In [58]:
##Rename columns
artists_table = artists_table.rename(columns={'artist_name':'artist'})
artists_table = artists_table.rename(columns={'ID':'artist_id'})
artists_table.head(3)

,artist_id,artist
0,1,Henri Salvador
1,2,Martin & Les Fées
2,3,Joseph Williams


### Import and normalize country table from DB

In [59]:
##Read table
country_table = pd.read_sql('country', engine)

In [60]:
##Rename columns
country_table = country_table.rename(columns={'ID':'country_id'})
country_table.head(3)

,country_id,country_name,alpha_2_code
0,1,Afghanistan,AF
1,2,Åland Islands,AX
2,3,Albania,AL


### Wrangle and match data

#### Jordan's top artist list

In [61]:
top_artist_jordan = pd.read_csv('../datasets/lastfm_api/top_artist_jordan.csv')
top_artist_jordan.head(3)

,Unnamed: 0,rank,artist,country
0,0,1,Lana Del Rey,Jordan
1,1,2,Pink Floyd,Jordan
2,2,3,Queen,Jordan


In [62]:
## Check country name in country table
country_table[country_table['country_name'] == 'Jordan']

,country_id,country_name,alpha_2_code
111,112,Jordan,JO


In [63]:
## Title case artists for artist matching
top_artist_jordan
top_artist_jordan['artist'] = top_artist_jordan['artist'].apply(lambda x : x.title())

##Find missing artists
artist_jordan_list = list(top_artist_jordan.artist)
artists_table_list = list(artists_table.artist)
for i in artist_jordan_list:
    if i not in artists_table_list:
        print(i)

Fairuz
Frank Sinatra


#### Switzerland's top artitst list

In [64]:
top_artist_switzerland = pd.read_csv('../datasets/lastfm_api/top_artist_switzerland.csv')
top_artist_switzerland.head(3)

,Unnamed: 0,rank,artist,country
0,0,1,Coldplay,Switzerland
1,1,2,Queen,Switzerland
2,2,3,David Bowie,Switzerland


In [65]:
## Check country name in country table
country_table[country_table['country_name'] == 'Switzerland']

,country_id,country_name,alpha_2_code
212,213,Switzerland,CH


In [66]:
## Title case artists for artist matching
top_artist_switzerland
top_artist_switzerland['artist'] = top_artist_switzerland['artist'].apply(lambda x : x.title())

##Find missing artists
artist_switzerland_list = list(top_artist_switzerland.artist)
artists_table_list = list(artists_table.artist)
for i in artist_switzerland_list:
    if i not in artists_table_list:
        print(i)

Mumford & Sons


#### Wrangling Spain's top artist list

In [67]:
top_artist_spain = pd.read_csv('../datasets/lastfm_api/top_artist_spain.csv')
top_artist_spain.head(3)

,Unnamed: 0,rank,artist,country
0,0,1,Radiohead,Spain
1,1,2,David Bowie,Spain
2,2,3,Kanye West,Spain


In [68]:
## Check country name in country table
country_table[country_table['country_name'] == 'Spain']

,country_id,country_name,alpha_2_code
205,206,Spain,ES


In [69]:
## Title case artists for artist matching
top_artist_spain
top_artist_spain['artist'] = top_artist_spain['artist'].apply(lambda x : x.title())

##Find missing artists
artist_spain_list = list(top_artist_spain.artist)
artists_table_list = list(artists_table.artist)
for i in artist_spain_list:
    if i not in artists_table_list:
        print(i)

#### Find parcial coincidences in artist table for missing artists:

* Fairuz
* Frank Sinatra
* Mumford & Sons

In [70]:
## Fairuz
artists_table.loc[artists_table.artist.str.contains("iru")]

,artist_id,artist
3792,3793,Michiru Oshima
4218,4219,Beirut
6098,6099,Pirulo Y La Tribu
9528,9529,Natiruts


In [71]:
## Frank Sinatra
artists_table.loc[artists_table.artist.str.contains("inatra")]

,artist_id,artist
4307,4308,Nancy Sinatra


In [72]:
## Mumford & Sons
artists_table.loc[artists_table.artist.str.contains("Sons")]

,artist_id,artist
1228,1229,Sons Of Bill
1271,1272,Rival Sons
4115,4116,Sons Of The East
6949,6950,The Glorious Sons
7951,7952,Young Rising Sons
8856,8857,Sons Of Zion
9236,9237,Sons Of Apollo
10848,10849,Summers Sons
11173,11174,Sons Of Kemet
11699,11700,The Arrogant Sons Of Bitches


These artists don't exist in our current DB. We need to add them.

#### Artists list update on DB

In [73]:
##Create DataFrame with the missing artists
new_artists = ['Mumford & Sons', 'Fairuz', 'Frank Sinatra']
df_new_artists = pd.DataFrame(new_artists)
df_new_artists.columns = ['artist_name']

In [74]:
##Add the missing artists to the DB table
df_new_artists.to_sql('artist', con=engine, if_exists = 'append', index = False) 

In [75]:
##Import table with new added artists
new_artists_table = pd.read_sql('artist', engine)

### Create new top charts table

In [76]:
##Create a unique DF with all countries
top_artists = pd.concat([top_artist_jordan, top_artist_switzerland, top_artist_spain], axis=0)
top_artists.drop('Unnamed: 0', axis=1, inplace = True)

In [77]:
##Add the country_id to the top_artists table
top_artists = top_artists.rename(columns={'country':'country_name'})
top_artists = pd.merge(top_artists, country_table, on='country_name')

In [78]:
##Add the artist_id to the top_artists table
top_artists = top_artists.rename(columns={'artist':'artist_name'})
new_artists_table = new_artists_table.rename(columns={'ID':'artist_id'})
top_artists = pd.merge(top_artists, new_artists_table, on='artist_name')

In [79]:
top_artists.head(3)

,rank,artist_name,country_name,country_id,alpha_2_code,artist_id
0,1,Lana Del Rey,Jordan,112,JO,2357
1,20,Lana Del Rey,Switzerland,213,CH,2357
2,18,Lana Del Rey,Spain,206,ES,2357


In [80]:
##Drop unnecessary columns
top_artists.drop(['alpha_2_code', 'artist_name', 'country_name'], axis=1, inplace = True)

In [81]:
## Export it to MySQL
top_artists.columns = ['ranking', 'country_id', 'artist_id']
top_artists.to_sql('top_artists_country', con=engine, if_exists = 'append', index = False) 

In [82]:
##Import it back with the table PK
top_artists_with_id = pd.read_sql('top_artists_country', engine)
print('Len top artists table after DB: ', len(top_artists_with_id))

Len top artists table after DB:  150
